In [36]:
import torch
import os
import copy
import random
import timm
import wandb
import cv2
import torchmetrics

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch.nn as nn
import pytorch_lightning as pl
import torchvision.transforms as transforms
import torch.optim.lr_scheduler as lr_scheduler

from adamp import AdamP
from datetime import datetime, timezone, timedelta
from PIL import Image
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from ipywidgets import interact

from torch.utils.data import DataLoader, Dataset
from torchmetrics.functional import accuracy, f1_score, precision, recall
# from sklearn.metrics import accuracy_score, f1_score

from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from sklearn.model_selection import KFold, StratifiedKFold, GroupKFold
from pytorch_lightning.loggers import WandbLogger

In [37]:
# 프로젝트 경로
PROJECT_DIR = './apple/'
# os.chdir(PROJECT_DIR)

# 데이터 경로
DATA_DIR = os.path.join(PROJECT_DIR, 'DATA')
TRAIN_IMG_DIR = os.path.join(DATA_DIR, 'train')
TRAIN_LABEL_DIR = os.path.join(DATA_DIR, 'train.csv')
TEST_IMG_DIR = os.path.join(DATA_DIR, 'test')
TEST_LABEL_DIR = os.path.join(DATA_DIR, 'test.csv')
SAMPLE_DIR = os.path.join(DATA_DIR, 'sample_submission.csv')

In [38]:
# 파일 수 확인 
print(len(os.listdir(TRAIN_IMG_DIR)))
print(len(os.listdir(TEST_IMG_DIR)))

10000
5000


In [39]:
RANDOM_SEED = 42
torch.manual_seed(RANDOM_SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)

In [40]:
# GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [41]:
class TestDataset(Dataset):
    def __init__(self, img_folder, label_path):
        self.df = pd.read_csv(label_path, usecols=['img_id'], dtype={'a_type':str})
        # self.df = label_path
        self.img_folder = img_folder
        self.transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
        ])
        self.img_ids = list(self.df['img_id'])

    def __len__(self):
        return len(self.img_ids)

    def __getitem__(self, index):
        image_path = os.path.join(self.img_folder, self.img_ids[index])
        img = Image.open(image_path)
        img = self.transform(img)
        img_ids = self.img_ids[index]

        return img, img_ids

In [42]:
class EfficientnetB4(pl.LightningModule):
    def __init__(self, args=None, optimizer='adam', scheduler='reducelr'):
        super().__init__()

        self.model = timm.create_model('efficientnet_b4', pretrained=True)
        self.model.classifier = nn.Sequential(
        nn.Linear(in_features = 1792, out_features = 625),
        nn.ReLU(),
        nn.Dropout(p=0.3),
        nn.Linear(625,256),
        nn.ReLU(),
        nn.Linear(256, 4)
        )

        self.args = args
        self.criterion = nn.CrossEntropyLoss()
        self.optimizer = optimizer
        self.scheduler = scheduler    

    def forward(self, x):
        output = self.model(x)
        return output

In [43]:
class InceptionV3(pl.LightningModule):
    def __init__(self, args=None, optimizer='adam', scheduler='reducelr'):
        super().__init__()
        self.model = timm.create_model('inception_v3', pretrained=True, num_classes=4)
        self.args = args
        self.criterion = nn.CrossEntropyLoss()
        self.optimizer = optimizer
        self.scheduler = scheduler    

    def forward(self, x):
        output = self.model(x)
        return output

In [44]:
class SeResnext26(pl.LightningModule):
    def __init__(self, args=None, optimizer='adam', scheduler='reducelr'):
        super().__init__()
        self.model = timm.create_model('seresnext26d_32x4d', pretrained=True, num_classes=4)
        self.args = args
        self.criterion = nn.CrossEntropyLoss()
        self.optimizer = optimizer
        self.scheduler = scheduler    

    def forward(self, x):
        output = self.model(x)
        return output

In [45]:
class Ensemble(pl.LightningModule):
    def __init__(self, model1, model2, model3, args=None, optimizer='adam', scheduler='reducelr'):
        super(Ensemble, self).__init__()
        self.model1 = model1
        self.model2 = model2
        self.model3 = model3

        self.model1.fc = nn.Identity()
        self.model2.fc = nn.Identity()
        self.model3.fc = nn.Identity()

        self.classifier = nn.Linear(8, 4)

        self.args = args
        self.criterion = nn.CrossEntropyLoss()
        self.optimizer = optimizer
        self.scheduler = scheduler    

    def forward(self, x):
        # output = self.model(x)
        x1 = self.model1(x.clone())
        x1 = x1.view(x1.size(0), -1)
        x2 = self.model2(x)
        x2 = x2.view(x2.size(0), -1)
        x3 = self.model3(x)
        x3 = x3.view(x3.size(0), -1)

        x = torch.cat((x2, x3), dim=1)
        output = self.classifier(x)
        return output


    def configure_optimizers(self):
        if self.optimizer == 'adam':
            optimizer = torch.optim.Adam(self.parameters(), lr=0.001)
        elif self.args.optimizer == 'adamw':
            optimizer = torch.optim.AdamW(self.parameters(), lr=0.001)
        elif self.args.optimizer == 'adamp':
            optimizer = AdamP(self.parameters(), lr=0.001, betas=(0.9, 0.999), weight_decay=1e-2)

        if self.scheduler == "reducelr":
            scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, patience=5, factor=0.5, mode="max", verbose=True)
            return {"optimizer": optimizer, "lr_scheduler": scheduler, "monitor": "val/f1"}

        elif self.scheduler == "cosineanneal":
            scheduler = lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=10, T_mult=1, eta_min=1e-5,
                                                                 last_epoch=-1, verbose=True)

        return {"optimizer": optimizer, "lr_scheduler": scheduler}


    def training_step(self, train_batch, batch_idx):
        image, label = train_batch

        outputs = self.forward(image)
        loss = self.criterion(outputs, label)
        acc = accuracy(outputs, label)
        f1 = f1_score(outputs, label, num_classes=4, average="macro")
        self.log("train/acc", acc, on_epoch=True, on_step=True, prog_bar=True)
        self.log("train/loss", loss, on_epoch=True, on_step=True, prog_bar=True, sync_dist=True)
        self.log("train/f1", f1, on_epoch=True, on_step=True, prog_bar=True, sync_dist=True)

        return {"acc":acc, "loss":loss, "f1-score":f1}


    def validation_step(self, train_batch, batch_idx):
        image, label = train_batch

        outputs = self.forward(image)
        loss = self.criterion(outputs, label)
        acc = accuracy(outputs, label)
        f1 = f1_score(outputs, label, num_classes=4, average="macro")
        self.log("val/acc", acc, on_epoch=True, on_step=True, prog_bar=True)
        self.log("val/loss", loss, on_epoch=True, on_step=True, prog_bar=True, sync_dist=True)
        self.log("val/f1", f1, on_epoch=True, on_step=True, prog_bar=True, sync_dist=True)

        return {"acc":acc, "loss":loss, "f1-score":f1}

In [46]:
model1 = EfficientnetB4()
model2 = InceptionV3()
model3 = SeResnext26()

model1 = model1.load_from_checkpoint('./apple/Models/effi.ckpt')
model2 = model2.load_from_checkpoint('./apple/Models/inception.ckpt')
model3 = model3.load_from_checkpoint('./apple/Models/res26.ckpt')

model1.to(device)
model2.to(device)
model3.to(device)

model1.eval()
model2.eval()
model3.eval()

SeResnext26(
  (model): ResNet(
    (conv1): Sequential(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU(inplace=True)
      (6): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    )
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act1): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stat

In [47]:
ensemble = Ensemble(model1=model1, model2=model2, model3=model3)
ensemble = ensemble.load_from_checkpoint('./apple/Models/ensemble3.ckpt', model1=model1, model2=model2, model3=model3)
ensemble.to(device)
ensemble.eval()

Ensemble(
  (model1): EfficientnetB4(
    (model): EfficientNet(
      (conv_stem): Conv2d(3, 48, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act1): SiLU(inplace=True)
      (blocks): Sequential(
        (0): Sequential(
          (0): DepthwiseSeparableConv(
            (conv_dw): Conv2d(48, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=48, bias=False)
            (bn1): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (act1): SiLU(inplace=True)
            (se): SqueezeExcite(
              (conv_reduce): Conv2d(48, 12, kernel_size=(1, 1), stride=(1, 1))
              (act1): SiLU(inplace=True)
              (conv_expand): Conv2d(12, 48, kernel_size=(1, 1), stride=(1, 1))
              (gate): Sigmoid()
            )
            (conv_pw): Conv2d(48, 24, kernel_size=(1, 1), stride=(1, 1), bias=False)
        

In [48]:
test_dataset = TestDataset(img_folder = TEST_IMG_DIR, label_path = os.path.join(TEST_LABEL_DIR))

# DataLoader
test_dataloader = DataLoader(dataset = test_dataset,
                              batch_size = 1,
                              num_workers = 0,
                              shuffle = False,
                              drop_last = False)

In [49]:
# 예측 진행
y_preds = []
img_ids = []

for batch_index, (x, img_id) in enumerate(tqdm(test_dataloader)):
    x = x.to(device, dtype=torch.float)
    y_logits = ensemble(x).cpu()
    y_pred = torch.argmax(y_logits, dim=1)
    y_pred = y_pred.cpu().tolist()
    img_ids.extend(img_id)
    y_preds.extend(y_pred)

100%|██████████| 5000/5000 [05:11<00:00, 16.06it/s]


In [50]:
pred_df = pd.DataFrame(list(zip(img_ids, y_preds)), columns=['img_id','a_type'])
label_decoding = {0:'HJ', 1:'HR', 2:'SG', 3:'AR'}
pred_df['a_type'] = pred_df['a_type'].replace(label_decoding)

In [51]:
sample_df = pd.read_csv(SAMPLE_DIR)
sorter = list(sample_df['img_id'])
resdf = pred_df.set_index('img_id')
result = resdf.loc[sorter].reset_index()

In [52]:
result.to_csv(os.path.join('./apple/results','ensemble3.csv'),index=False)